##1) Load and save UA-DETRAC

In [ ]:
!git clone https://github.com/Ahmad1Alzein/vehicle-speed-estimation.git
%cd /content/vehicle-speed-estimation
!git checkout feature-detection
!git pull


Cloning into 'vehicle-speed-estimation'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 52 (delta 21), reused 43 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 10.30 KiB | 10.30 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/vehicle-speed-estimation
Branch 'feature-detection' set up to track remote branch 'feature-detection' from 'origin'.
Switched to a new branch 'feature-detection'
Already up to date.


In [ ]:
!pip -q install kagglehub
!python src/scripts/download_uadetrac.py --out runtime/paths.json


100% 9.23G/9.23G [01:22<00:00, 120MB/s]
Extracting files...
UA-DETRAC ready at: /root/.cache/kagglehub/datasets/bratjay/ua-detrac-orig/versions/2
Path saved to: runtime/paths.json


In [ ]:
import os
import shutil
from tqdm import tqdm


def copy_dataset_to_content(
    source_path: str,
    target_root: str = "/content/datasets/uadetrac"
) -> str:
    if not os.path.exists(source_path):
        raise FileNotFoundError(f"Source dataset path not found: {source_path}")

    os.makedirs(target_root, exist_ok=True)

    # Collect all files first (for tqdm total)
    all_files = []
    for root, _, files in os.walk(source_path):
        for f in files:
            src_file = os.path.join(root, f)
            rel_path = os.path.relpath(src_file, source_path)
            dst_file = os.path.join(target_root, rel_path)
            all_files.append((src_file, dst_file))

    # Copy with progress bar
    for src_file, dst_file in tqdm(all_files, desc="Copying dataset", unit="file"):
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        shutil.copy2(src_file, dst_file)

    print("Dataset copied to:", target_root)
    return target_root


In [ ]:
import json

PATHS_JSON = "runtime/paths.json"
TARGET_ROOT = "/content/datasets/uadetrac"
if not os.path.exists(PATHS_JSON):
        raise FileNotFoundError(f"{PATHS_JSON} not found")
with open(PATHS_JSON, "r") as f:
  data = json.load(f)
dataset_path = data.get("ua_detrac_root")
print(dataset_path)
copy_dataset_to_content(dataset_path,TARGET_ROOT)

/root/.cache/kagglehub/datasets/bratjay/ua-detrac-orig/versions/2


Copying dataset: 100%|██████████| 141039/141039 [02:42<00:00, 870.55file/s]

Dataset copied to: /content/datasets/uadetrac


'/content/datasets/uadetrac'

##2) Feature Extraction

In [ ]:
!pip -q install pillow

In [ ]:
!python src/scripts/uadetrac_to_yolo.py \
  --split train \
  --images_root "/content/datasets/uadetrac/DETRAC-Images/DETRAC-Images" \
  --ann_root "/content/datasets/uadetrac/DETRAC-Train-Annotations-XML/DETRAC-Train-Annotations-XML" \
  --out_root "/content/datasets/uadetrac_yolo" \
  --val_ratio 0.2


[INFO] MVI_39031 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39051 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39211 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39271 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39311 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39361 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39371 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39401 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39501 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_39511 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_40701 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_40711 has no annotation in DETRAC-Train-Annotations-XML, skipping.
[INFO] MVI_40712 has no annotation in DETRAC-Train-Annotations-X

In [ ]:
!python src/scripts/uadetrac_to_yolo.py \
  --split test \
  --images_root "/content/datasets/uadetrac/DETRAC-Images/DETRAC-Images" \
  --ann_root "/content/datasets/uadetrac/DETRAC-Test-Annotations-XML/DETRAC-Test-Annotations-XML" \
  --out_root "/content/datasets/uadetrac_test"


[INFO] MVI_20011 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20012 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20032 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20033 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20034 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20035 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20051 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20052 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20061 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20062 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20063 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20064 has no annotation in DETRAC-Test-Annotations-XML, skipping.
[INFO] MVI_20065 has no annotation in DETRAC-Test-Annotations-XML, skipping.

##3) Train with YOLOv8m.pt

In [ ]:
!pip install -q ultralytics pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.3 MB/s eta 0:00:00


In [ ]:
!python src/detection/train.py \
  --data "/content/datasets/uadetrac_yolo/data.yaml" \
  --model "yolov8s.pt" \
  --epochs 5 \
  --imgsz 640 \
  --batch 16 \
  --project "/content/runs" \
  --name "uadetrac_yolov8s"



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/uadetrac_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=

In [ ]:
!rm -rf /content/runs/uadetrac_yolov8s

###🔁 Resume later (choose how many epoch)

Upload only last.pt (if the runtime disconnected and the zip before downloaded)
Upload last.pt directly, then place it here:

In [ ]:
!mkdir -p /content/runs
!unzip /content/uadetrac_yolov8s_run_e10.zip -d /content/runs

Archive:  /content/uadetrac_yolov8s_run_e10.zip
   creating: /content/runs/uadetrac_yolov8s2/
  inflating: /content/runs/uadetrac_yolov8s2/val_batch2_labels.jpg  
  inflating: /content/runs/uadetrac_yolov8s2/val_batch0_pred.jpg  
  inflating: /content/runs/uadetrac_yolov8s2/results.png  
  inflating: /content/runs/uadetrac_yolov8s2/val_batch0_labels.jpg  
  inflating: /content/runs/uadetrac_yolov8s2/BoxF1_curve.png  
   creating: /content/runs/uadetrac_yolov8s2/weights/
  inflating: /content/runs/uadetrac_yolov8s2/weights/best.pt  
  inflating: /content/runs/uadetrac_yolov8s2/weights/last.pt  
  inflating: /content/runs/uadetrac_yolov8s2/val_batch2_pred.jpg  
  inflating: /content/runs/uadetrac_yolov8s2/confusion_matrix_normalized.png  
  inflating: /content/runs/uadetrac_yolov8s2/BoxP_curve.png  
  inflating: /content/runs/uadetrac_yolov8s2/results.csv  
  inflating: /content/runs/uadetrac_yolov8s2/args.yaml  
  inflating: /content/runs/uadetrac_yolov8s2/BoxR_curve.png  
  inflating: 

In [ ]:
##try if this cell alone works and then train resumed (if not so run the below cells)
import torch

pt = "/content/runs/uadetrac_yolov8s/weights/last.pt"

ckpt = torch.load(pt, map_location="cpu", weights_only=False)

# 1) Decide what you ACTUALLY trained
LAST_EPOCH_COMPLETED = 10   # <-- change if needed

# 2) Fix epoch (0-based index)
ckpt["epoch"] = LAST_EPOCH_COMPLETED - 1

# 3) Ensure target total epochs is higher
ckpt.setdefault("train_args", {})
ckpt["train_args"]["epochs"] = max(
    ckpt["train_args"].get("epochs", 0),
    LAST_EPOCH_COMPLETED + 5
)

torch.save(ckpt, pt)

print("✅ Fixed checkpoint")
print("epoch =", ckpt["epoch"])
print("target epochs =", ckpt["train_args"]["epochs"])


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Fixed checkpoint
epoch = 9
target epochs = 15


In [ ]:
!python src/detection/train.py \
  --data "/content/datasets/uadetrac_yolo/data.yaml" \
  --model "/content/runs/uadetrac_yolov8s/weights/last.pt" \
  --epochs 10 \
  --imgsz 640 \
  --batch 16 \
  --project "/content/runs" \
  --name "uadetrac_yolov8s" \
  --resume


Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/uadetrac_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/uadetrac_yolov8s/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=uadetrac_yolov8s2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap

In [ ]:
# Create export directory
!mkdir -p /content/export
# Zip the FULL run folder into /content/export
!cd /content/runs && zip -r /content/export/uadetrac_yolov8s_run_e15.zip uadetrac_yolov8s2
from google.colab import files
files.download("/content/export/uadetrac_yolov8s_run_e15.zip")


  adding: uadetrac_yolov8s2/ (stored 0%)
  adding: uadetrac_yolov8s2/val_batch2_labels.jpg (deflated 6%)
  adding: uadetrac_yolov8s2/val_batch0_pred.jpg (deflated 5%)
  adding: uadetrac_yolov8s2/results.png (deflated 8%)
  adding: uadetrac_yolov8s2/val_batch0_labels.jpg (deflated 6%)
  adding: uadetrac_yolov8s2/BoxF1_curve.png (deflated 11%)
  adding: uadetrac_yolov8s2/weights/ (stored 0%)
  adding: uadetrac_yolov8s2/weights/best.pt (deflated 8%)
  adding: uadetrac_yolov8s2/weights/last.pt (deflated 8%)
  adding: uadetrac_yolov8s2/val_batch2_pred.jpg (deflated 5%)
  adding: uadetrac_yolov8s2/confusion_matrix_normalized.png (deflated 24%)
  adding: uadetrac_yolov8s2/BoxP_curve.png (deflated 12%)
  adding: uadetrac_yolov8s2/results.csv (deflated 54%)
  adding: uadetrac_yolov8s2/args.yaml (deflated 55%)
  adding: uadetrac_yolov8s2/BoxR_curve.png (deflated 10%)
  adding: uadetrac_yolov8s2/confusion_matrix.png (deflated 24%)
  adding: uadetrac_yolov8s2/val_batch1_pred.jpg (deflated 6%)
  ad

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q deep-sort-realtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.0 MB/s eta 0:00:00
